## Adding new features to the CTG dataset using the following sources:
1. AACT - a publicly available relational database that contains all information (protocol and result data elements) about every study registered in ClinicalTrials.gov.: https://aact.ctti-clinicaltrials.org/
2.  Pharma 50: The top pharma companies in the world for 2025: https://www.drugdiscoverytrends.com/pharma-50-the-50-largest-pharmaceutical-companies-in-the-world-for-2025/

#### New feature: number of groups
Based on AACT design_groups table

In [2]:
# Uploading neccessary columns from the CTG dataset
import pandas as pd
df=pd.read_csv('ctgAll_EDA1205_atcc_std_3005_3.csv', usecols=['NCT_Number','Enrollment','Sponsor'])
df.shape

(436180, 3)

In [3]:
groups=pd.read_csv('design_groups0520.csv',usecols=['design_group_id', 'nct_id', 'group_type'])
print(groups.shape)
print(groups.columns)

(978499, 3)
Index(['design_group_id', 'nct_id', 'group_type'], dtype='object')


In [4]:
# Step 1: Add 'No_of_Group' column by counting design_group_id per nct_id
groups['No_of_Groups'] = groups.groupby('nct_id')['design_group_id'].transform('count')

# Step 2: Deduplicate on 'nct_id' and rename to 'NCT_Number'
groups_unique = groups.drop_duplicates(subset='nct_id').rename(columns={'nct_id': 'NCT_Number'})

groups_unique.head()


,design_group_id,NCT_Number,group_type,No_of_Groups
0,131079376,NCT02089971,NaN,1
1,131079400,NCT05827666,ACTIVE_COMPARATOR,3
4,131079410,NCT04384679,EXPERIMENTAL,2
6,131079412,NCT04086979,EXPERIMENTAL,2
8,131079424,NCT04294979,EXPERIMENTAL,1


In [5]:
# Step 3: Merge with main df 
merged = df.merge(groups_unique[['NCT_Number', 'No_of_Groups']], on='NCT_Number', how='left')
merged['No_of_Groups'] = merged['No_of_Groups'].fillna(1)
merged.head()

,NCT_Number,Sponsor,Enrollment,No_of_Groups
0,NCT00517179,"Hospital Authority, Hong Kong",40.0,1.0
1,NCT00812279,Philip Morris Products S.A.,112.0,3.0
2,NCT03878979,Sidney Kimmel Comprehensive Cancer Center at J...,26.0,2.0
3,NCT03058679,University of Pennsylvania,197.0,2.0
4,NCT03960879,Lei Li,300.0,1.0


In [6]:
# Step 4: Create 'Group_Size' column by dividing 'Enrollment' by 'No_of_Group'
merged['Group_Size'] = merged['Enrollment'] / merged['No_of_Groups']
merged.head(5)

,NCT_Number,Sponsor,Enrollment,No_of_Groups,Group_Size
0,NCT00517179,"Hospital Authority, Hong Kong",40.0,1.0,40.000000
1,NCT00812279,Philip Morris Products S.A.,112.0,3.0,37.333333
2,NCT03878979,Sidney Kimmel Comprehensive Cancer Center at J...,26.0,2.0,13.000000
3,NCT03058679,University of Pennsylvania,197.0,2.0,98.500000
4,NCT03960879,Lei Li,300.0,1.0,300.000000


#### New feature: Does the Sponsor belong to the list of "big pharma" companies 
Based on Pharma 50 webpage

In [7]:
# Step 1: Define a list of Big Pharma companies
big_pharma_list = [
    "Pfizer", "Johnson & Johnson", "Roche", "Novartis", "Merck", "Sanofi", "AstraZeneca", 
    "GlaxoSmithKline", "Bristol-Myers Squibb", "AbbVie", "Amgen", "Eli Lilly", "Bayer", "Takeda"
]

# Normalize names for better matching
big_pharma_normalized = [name.lower() for name in big_pharma_list]

# Step 2: Create a normalized version of sponsor names in the main dataframe
merged['Sponsor_clean'] = merged['Sponsor'].str.lower().str.strip()

# Step 3: Check if sponsor is in Big Pharma list
merged['Is_Big_Pharma'] = merged['Sponsor_clean'].apply(
    lambda x: any(bp in x for bp in big_pharma_normalized)
)
merged.head()

,NCT_Number,Sponsor,Enrollment,No_of_Groups,Group_Size,Sponsor_clean,Is_Big_Pharma
0,NCT00517179,"Hospital Authority, Hong Kong",40.0,1.0,40.000000,"hospital authority, hong kong",False
1,NCT00812279,Philip Morris Products S.A.,112.0,3.0,37.333333,philip morris products s.a.,False
2,NCT03878979,Sidney Kimmel Comprehensive Cancer Center at J...,26.0,2.0,13.000000,sidney kimmel comprehensive cancer center at j...,False
3,NCT03058679,University of Pennsylvania,197.0,2.0,98.500000,university of pennsylvania,False
4,NCT03960879,Lei Li,300.0,1.0,300.000000,lei li,False


#### New Features: disease category
To add a disease-specific category (e.g., cancer), we conducted a dedicated search on ClinicalTrials.gov using relevant condition keywords. This process yielded a list of NCT IDs corresponding to trials related to the condition of interest. We saved this list in a separate file and then merged it with our main dataset to create a binary indicator feature

In [8]:
cancer=pd.read_csv('cancer.csv')
cancer.shape

(114646, 1)

In [9]:
# Merge with indicator to see which rows matched
df1 = merged.merge(cancer[['NCT_Number']], on='NCT_Number', how='left', indicator='cancer_flag')

# Create Is_Cancer column: True if matched, else False
df1['Is_Cancer'] = df1['cancer_flag'] == 'both'

# Drop the helper column if not needed
df1 = df1.drop(columns=['cancer_flag'])
df1.head(1)


,NCT_Number,Sponsor,Enrollment,No_of_Groups,Group_Size,Sponsor_clean,Is_Big_Pharma,Is_Cancer
0,NCT00517179,"Hospital Authority, Hong Kong",40.0,1.0,40.0,"hospital authority, hong kong",False,False


In [10]:
df1.Is_Cancer.value_counts()

Is_Cancer
False    355087
True      81093
Name: count, dtype: int64

In [11]:
Neurological=pd.read_csv('Neurological.csv')
# Merge with the indicator to see which rows matched
df2 = df1.merge(Neurological['NCT_Number'], on='NCT_Number', how='left', indicator='Neurological_flag')


In [12]:
# Create Is_Neurological column: True if matched, else False
df2['Is_Neurological'] = df2['Neurological_flag'] == 'both'

# Drop the helper column if not needed
df2 = df2.drop(columns=['Neurological_flag'])
df2.Is_Neurological.value_counts()

Is_Neurological
False    404038
True      32142
Name: count, dtype: int64

In [13]:
Infectious=pd.read_csv('Infectious.csv')
# Merge with indicator to see which rows matched
df3 = df2.merge(Infectious['NCT_Number'], on='NCT_Number', how='left', indicator='Infectious_flag')


In [14]:
# Create Is_Neurological column: True if matched, else False
df3['Is_Infectious'] = df3['Infectious_flag'] == 'both'

# Drop the helper column if not needed
df3 = df3.drop(columns=['Infectious_flag'])
df3.Is_Infectious.value_counts()

Is_Infectious
False    416859
True      19321
Name: count, dtype: int64

In [15]:
Immunological =pd.read_csv('Immunological.csv')
# Merge with indicator to see which rows matched
df4 = df3.merge(Immunological['NCT_Number'], on='NCT_Number', how='left', indicator='Immunological_flag')
# Create Is_Immunological column: True if matched, else False
df4['Is_Immunological'] = df4['Immunological_flag'] == 'both'

# Drop the helper column if not needed
df4 = df4.drop(columns=['Immunological_flag'])
df4.Is_Immunological.value_counts()



Is_Immunological
False    395588
True      40592
Name: count, dtype: int64

In [16]:

Hematological =pd.read_csv('Hematology.csv')
# Merge with indicator to see which rows matched
df5 = df4.merge(Hematological['NCT_Number'], on='NCT_Number', how='left', indicator='Hematological_flag')
# Create Is_Hematological column: True if matched, else False
df5['Is_Hematological'] = df5['Hematological_flag'] == 'both'

# Drop the helper column if not needed
df5 = df5.drop(columns=['Hematological_flag'])
df5.Is_Hematological.value_counts()

Is_Hematological
False    421274
True      14906
Name: count, dtype: int64

In [17]:
Vaccines =pd.read_csv('Vaccines.csv')
# Merge with indicator to see which rows matched
df6 = df5.merge(Vaccines['NCT_Number'], on='NCT_Number', how='left', indicator='Vaccines_flag')
# Create Is_Vaccines column: True if matched, else False
df6['Is_Vaccine'] = df6['Vaccines_flag'] == 'both'

# Drop the helper column if not needed
df6 = df6.drop(columns=['Vaccines_flag'])
df6.Is_Vaccine.value_counts()

Is_Vaccine
False    428260
True       7920
Name: count, dtype: int64

In [18]:
Respiratory =pd.read_csv('Respiratory.csv')
# Merge with indicator to see which rows matched
df7 = df6.merge(Respiratory['NCT_Number'], on='NCT_Number', how='left', indicator='Respiratory_flag')
# Create Is_Respiratory column: True if matched, else False
df7['Is_Respiratory'] = df7['Respiratory_flag'] == 'both'

# Drop the helper column if not needed
df7 = df7.drop(columns=['Respiratory_flag'])
df7.Is_Respiratory.value_counts()

Is_Respiratory
False    393790
True      42390
Name: count, dtype: int64

In [19]:
Metabolic =pd.read_csv('Metabolic.csv')
# Merge with indicator to see which rows matched
df8 = df7.merge(Metabolic['NCT_Number'], on='NCT_Number', how='left', indicator='Metabolic_flag')
# Create Is_Metabolic column: True if matched, else False
df8['Is_Metabolic'] = df8['Metabolic_flag'] == 'both'

# Drop the helper column if not needed
df8 = df8.drop(columns=['Metabolic_flag'])
df8.Is_Metabolic.value_counts()

Is_Metabolic
False    405107
True      31073
Name: count, dtype: int64

In [20]:
Cardiovascular = pd.read_csv('Cardiovascular.csv')
# Merge with indicator to see which rows matched
df9 = df8.merge(Cardiovascular['NCT_Number'], on='NCT_Number', how='left', indicator='Cardiovascular_flag')
# Create Is_Cardiovascular column: True if matched, else False
df9['Is_Cardiovascular'] = df9['Cardiovascular_flag'] == 'both'

# Drop the helper column if not needed
df9 = df9.drop(columns=['Cardiovascular_flag'])
df9.Is_Cardiovascular.value_counts()

Is_Cardiovascular
False    384451
True      51729
Name: count, dtype: int64

In [21]:

Mental_Health = pd.read_csv('Mental Health.csv')
# Merge with indicator to see which rows matched
df10 = df9.merge(Mental_Health['NCT_Number'], on='NCT_Number', how='left', indicator='Mental_Health_flag')
# Create Is_Mental_Health column: True if matched, else False
df10['Is_Mental_Health'] = df10['Mental_Health_flag'] == 'both'

# Drop the helper column if not needed
df10 = df10.drop(columns=['Mental_Health_flag'])
df10.Is_Mental_Health.value_counts()

Is_Mental_Health
False    431789
True       4391
Name: count, dtype: int64

In [22]:
Congenital = pd.read_csv('Congenital.csv')
# Merge with indicator to see which rows matched
df11 = df10.merge(Congenital['NCT_Number'], on='NCT_Number', how='left', indicator='Congenital_flag')
# Create Is_Congenital column: True if matched, else False
df11['Is_Congenital'] = df11['Congenital_flag'] == 'both'

# Drop the helper column if not needed
df11 = df11.drop(columns=['Congenital_flag'])
df11.Is_Congenital.value_counts()

Is_Congenital
False    428471
True       7709
Name: count, dtype: int64

In [23]:
Injuries = pd.read_csv('Injuries.csv')
# Merge with indicator to see which rows matched
df12 = df11.merge(Injuries['NCT_Number'], on='NCT_Number', how='left', indicator='Injuries_flag')
# Create Is_Injuries column: True if matched, else False
df12['Is_Injuries'] = df12['Injuries_flag'] == 'both'

# Drop the helper column if not needed
df12 = df12.drop(columns=['Injuries_flag'])
df12.Is_Injuries.value_counts()

Is_Injuries
False    412498
True      23682
Name: count, dtype: int64

In [24]:
Musculoskeletal = pd.read_csv('Musculoskeletal.csv')
# Merge with indicator to see which rows matched
df13 = df12.merge(Musculoskeletal['NCT_Number'], on='NCT_Number', how='left', indicator='Musculoskeletal_flag')
# Create Is_Musculoskeletal column: True if matched, else False
df13['Is_Musculoskeletal'] = df13['Musculoskeletal_flag'] == 'both'

# Drop the helper column if not needed
df13 = df13.drop(columns=['Musculoskeletal_flag'])
df13.Is_Musculoskeletal.value_counts()

Is_Musculoskeletal
False    412736
True      23444
Name: count, dtype: int64

In [25]:
Ocular= pd.read_csv('Ocular.csv')
# Merge with indicator to see which rows matched
df14 = df13.merge(Ocular['NCT_Number'], on='NCT_Number', how='left', indicator='Ocular_flag')
# Create Is_Ocular column: True if matched, else False
df14['Is_Ocular'] = df14['Ocular_flag'] == 'both'

# Drop the helper column if not needed
df14 = df14.drop(columns=['Ocular_flag'])
df14.Is_Ocular.value_counts()

Is_Ocular
False    423273
True      12907
Name: count, dtype: int64

In [26]:
Dermatological = pd.read_csv('Dermatological.csv')
# Merge with indicator to see which rows matched
df15 = df14.merge(Dermatological['NCT_Number'], on='NCT_Number', how='left', indicator='Dermatological_flag')
# Create Is_Dermatological column: True if matched, else False
df15['Is_Dermatological'] = df15['Dermatological_flag'] == 'both'

# Drop the helper column if not needed
df15 = df15.drop(columns=['Dermatological_flag'])
df15.Is_Dermatological.value_counts()


Is_Dermatological
False    412361
True      23819
Name: count, dtype: int64

In [27]:
Urological = pd.read_csv('Urological.csv')
# Merge with indicator to see which rows matched
df16 = df15.merge(Urological['NCT_Number'], on='NCT_Number', how='left', indicator='Urological_flag')
# Create Is_Urological column: True if matched, else False
df16['Is_Urological'] = df16['Urological_flag'] == 'both'

# Drop the helper column if not needed
df16 = df16.drop(columns=['Urological_flag'])
df16.Is_Urological.value_counts()


Is_Urological
False    421967
True      14213
Name: count, dtype: int64

In [28]:
Endocrinological= pd.read_csv('Endocrinological.csv')
# Merge with indicator to see which rows matched
df17 = df16.merge(Endocrinological['NCT_Number'], on='NCT_Number', how='left', indicator='Endocrinological_flag')
# Create Is_Endocrinological column: True if matched, else False
df17['Is_Endocrinological'] = df17['Endocrinological_flag'] == 'both'

# Drop the helper column if not needed
df17 = df17.drop(columns=['Endocrinological_flag'])
df17.Is_Endocrinological.value_counts()

Is_Endocrinological
False    412323
True      23857
Name: count, dtype: int64

In [29]:
Healthy= pd.read_csv('Healthy.csv')
# Merge with indicator to see which rows matched
df18 = df17.merge(Healthy['NCT_Number'], on='NCT_Number', how='left', indicator='Healthy_flag')
# Create Is_Healthy column: True if matched, else False
df18['Is_Healthy'] = df18['Healthy_flag'] == 'both'

# Drop the helper column if not needed
df18 = df18.drop(columns=['Healthy_flag'])
df18.Is_Healthy.value_counts()

Is_Healthy
False    420142
True      16038
Name: count, dtype: int64

In [30]:
Pain = pd.read_csv('Pain.csv')
# Merge with indicator to see which rows matched
df19 = df18.merge(Pain['NCT_Number'], on='NCT_Number', how='left', indicator='Pain_flag')
# Create Is_Pain column: True if matched, else False
df19['Is_Pain'] = df19['Pain_flag'] == 'both'

# Drop the helper column if not needed
df19 = df19.drop(columns=['Pain_flag'])
df19.Is_Pain.value_counts()

Is_Pain
False    407202
True      28978
Name: count, dtype: int64

In [31]:
df = pd.read_csv('ctgAll_EDA1205_atcc_std.csv')
df.columns

Index(['NCT_Number', 'Study_Title', 'Study_Status', 'Enrollment',
       'Start_date_formated', 'Completion_date_formated', 'Enrollment_log',
       'Start_Year', 'Completion_Year', 'Study_Duration_Capped',
       'MOA_mesh_term', 'primary_mesh_term', 'safety', 'efficacy',
       'retraction', 'study_success', 'Published_in_Pubmed',
       'Safety_Proportion', 'Efficacy_Proportion', 'Enrollment_std',
       'Enrollment_log_std', 'Start_Year_std', 'Completion_Year_std',
       'Study_Duration_Capped_std', 'Safety_Proportion_std',
       'Efficacy_Proportion_std', 'region_Simplified_East Asia',
       'region_Simplified_Europe', 'region_Simplified_North America',
       'region_Simplified_OTHER', 'Study_Status_Simplified_Completed',
       'Study_Status_Simplified_Ended_Early',
       'Study_Status_Simplified_Ongoing', 'Study_Status_Simplified_Unknown',
       'Sex_ALL', 'Sex_FEMALE', 'Sex_MALE', 'Age_Simplified_ADULT',
       'Age_Simplified_ALL', 'Age_Simplified_CHILD',
       'Age_Sim

In [32]:
# Dropping original, unstandardized columns and other success labels ('study_success', 'Published_in_Pubmed') to avoid leakage  
columns_to_drop=['Study_Title', 'Study_Status', 'Enrollment',
       'Start_date_formated','Completion_date_formated', 'Enrollment_log',
       'Start_Year', 'Study_Duration_Capped',
       'MOA_mesh_term', 'primary_mesh_term', 'safety', 'efficacy',
       'Published_in_Pubmed','Safety_Proportion', 'Efficacy_Proportion']
df= df.drop(columns=columns_to_drop)

In [33]:
df.to_csv('ctgAll_EDA1205_atcc_std_cleaned.csv', index=False)

In [34]:
#Merge back to the main CTG df
df = pd.read_csv('ctgAll_EDA1205_atcc_std_cleaned.csv')
df.columns
merge = df.merge(
    df19,
    on='NCT_Number',
    how='left',
    )

### New feature: subject retention rate
Based on data from the AACT 'droupout' table. dropput_grouped dataset contains the count of dropout incidences grouped by nct_id.

In [35]:
dropouts = pd.read_csv('dropput_grouped.csv')
dropouts.rename(columns={
    'nct_id':'NCT_Number',
     'number': 'Dropouts'
}, inplace=True)
dropouts.columns

Index(['NCT_Number', 'Dropouts'], dtype='object')

In [36]:
# Merging dropouts df with the CTG df
df20 = merge.merge(dropouts, on='NCT_Number', how='left')
df20['Dropouts'].isnull().sum()

133255

In [37]:
# Filling NaN values (studies in which no patients were dropped out) with 0 
df20['Dropouts'] = df20['Dropouts'].fillna(0)
# Calculating the retention rate, defined as (1-proportion of patients who dropped out from the overall patient enrollment)
df20['Retention_rate'] = 1 - (df20['Dropouts'] / df20['Enrollment'])
df20['Retention_rate'].isnull().sum()

0

### New feature: number of collaborators 
Based on data from the AACT 'Collaborators' table.

In [38]:
collaborators = pd.read_csv("Collaborators.csv", encoding='latin1')
# Count the number of collaborators separated by '|'
collaborators['num_collaborators'] = collaborators['Collaborators'].fillna('').apply(
    lambda x: len(x.split('|')) if x else 0
)
#  Merge with main df
df21 = df20.merge(collaborators, on='NCT_Number', how='left')

### New features: sponsor experience with the specific condition and with the management of Phase 3 studies
New features created from 'Sponsor', 'Condition' and 'Phases_Simplified' features of the CTG dataset

In [39]:
df = pd.read_csv('ctgAll_EDA1205.csv', usecols=['Conditions','Sponsor','NCT_Number','Start_Year','Phases_Simplified'])

# Ensure consistent formatting
df['Sponsor_clean'] = df['Sponsor'].str.lower().str.strip()
df['Conditions_clean'] = df['Conditions'].str.lower().str.strip()

# Sort by Start_Year
df = df.sort_values(by='Start_Year').copy()

# Create a cumulative count per Sponsor–Condition pair
df['SponsorCondition_experience'] = (
    df.groupby(['Sponsor_clean', 'Conditions_clean']).cumcount() + 1
)


In [40]:
# Sort data by Start_Year
df = df.sort_values(by='Start_Year').copy()

# Create a mask for Phase 3 trials
phase3_mask = df['Phases_Simplified'] == 'PHASE3'

# Initialize column with NaNs or zeros
df['Sponsor_Phase3_Experience'] = 0

# Compute cumulative Phase 3 experience for each sponsor
df.loc[phase3_mask, 'Sponsor_Phase3_Experience'] = (
    df[phase3_mask].groupby('Sponsor_clean').cumcount() + 1
)


In [41]:
# merge with main df
df22 = df21.merge(
    df[['NCT_Number', 'SponsorCondition_experience', 'Sponsor_Phase3_Experience']],
    on='NCT_Number',
    how='left'
)

## Standartization of new numerical features

In [42]:
from sklearn.preprocessing import StandardScaler

# Fill NaNs
df22['No_of_Groups'] = df22['No_of_Groups'].fillna(1)
df22['Group_Size'] = df22['Group_Size'].fillna(df22['Enrollment'])

# Select relevant columns
columns_to_scale = ['num_collaborators', 'SponsorCondition_experience',
       'Sponsor_Phase3_Experience','Retention_rate','No_of_Groups','Group_Size']

# Create scalers
scaler_standard = StandardScaler()

# Fit and transform the data
df_standardized = pd.DataFrame(scaler_standard.fit_transform(df22[columns_to_scale]),
                               columns=[col + '_std' for col in columns_to_scale])


# Join back to main DataFrame
df = pd.concat([df22, df_standardized], axis=1)

In [43]:
# Dropping original, unstandardized columns   
columns_to_drop=['num_collaborators', 'SponsorCondition_experience',
       'Sponsor_Phase3_Experience','Retention_rate','No_of_Groups','Group_Size']
df= df.drop(columns=columns_to_drop)

In [46]:
df.to_csv('CTG_Extra_features.csv', index=False)